# Best location to open an Italian Restaurant in Toronto

# Code <br>
# 1.Scrape Wikipedia table


In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
wiki_url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
page = requests.get(wiki_url)
soup = BeautifulSoup(page.text, 'html.parser')
#print(soup)

In [5]:
my_table = soup.find_all('table',attrs={'class':'wikitable sortable'})
#print(my_table)

In [6]:
df = pd.read_html(str(my_table))
pd.set_option('display.max_columns',20)
pd.set_option('display.max_colwidth',30)
pd.set_option('display.max_rows',10)
print(df)

[    Postal Code           Borough                   Neighborhood
0           M1A      Not assigned                   Not assigned
1           M2A      Not assigned                   Not assigned
2           M3A        North York                      Parkwoods
3           M4A        North York               Victoria Village
4           M5A  Downtown Toronto      Regent Park, Harbourfront
..          ...               ...                            ...
175         M5Z      Not assigned                   Not assigned
176         M6Z      Not assigned                   Not assigned
177         M7Z      Not assigned                   Not assigned
178         M8Z         Etobicoke  Mimico NW, The Queensway W...
179         M9Z      Not assigned                   Not assigned

[180 rows x 3 columns]]


# 2.Data cleaning

In [7]:
df1 = df[0]

#print ((df1['Borough'])!= 'Not assigned')
df2 = df1[df1.Borough != 'Not assigned']
#df2 = df1[df1.Borough != 'nan']
df3 = df2.iloc[:,0:3]
print(df3)

    Postal Code           Borough                   Neighborhood
2           M3A        North York                      Parkwoods
3           M4A        North York               Victoria Village
4           M5A  Downtown Toronto      Regent Park, Harbourfront
5           M6A        North York  Lawrence Manor, Lawrence H...
6           M7A  Downtown Toronto  Queen's Park, Ontario Prov...
..          ...               ...                            ...
160         M8X         Etobicoke  The Kingsway, Montgomery R...
165         M4Y  Downtown Toronto           Church and Wellesley
168         M7Y      East Toronto  Business reply mail Proces...
169         M8Y         Etobicoke  Old Mill South, King's Mil...
178         M8Z         Etobicoke  Mimico NW, The Queensway W...

[103 rows x 3 columns]


In [8]:
df3.shape

(103, 3)

# 3.Segmenting and Clustering Neighborhoods

In [9]:
df4 = pd.DataFrame(df3)
df4

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence H..."
6,M7A,Downtown Toronto,"Queen's Park, Ontario Prov..."
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery R..."
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Proces...
169,M8Y,Etobicoke,"Old Mill South, King's Mil..."


In [10]:
import numpy as np 
import json 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


In [11]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_92b1a7b939944c14add0427ffeb79e45 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='i0kMTJQSmt-5qEoBz_WjpQ8qpAxr5L9UN0gLLnzFnF4p',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_92b1a7b939944c14add0427ffeb79e45.get_object(Bucket='bestlocationtoopenanitalianrestau-donotdelete-pr-x9i95mct7smnmp',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_csv(body)
df_data_0.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_both = pd.merge(df4,df_data_0, on='Postal Code')
df_both

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence H...",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Prov...",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery R...",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Proces...,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mil...",43.636258,-79.498509


In [13]:
!pip install geopy
import geopy
!pip install folium
import folium 
print('Imported')

     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011
Imported


In [14]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent = "Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(df_both['Latitude'], df_both['Longitude'], df_both['Borough'], df_both['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version 

In [16]:
CLIENT_ID = 'FP2YLWL5I4PV3ZST40Q2KFUSE5NP35FVVDP5APVEO1WKDACK' # your Foursquare ID
CLIENT_SECRET = '0UFI4D1UXNX50NIBJKGTLT4ML2JRA10ACFWGI4MP2JCCKQIV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FP2YLWL5I4PV3ZST40Q2KFUSE5NP35FVVDP5APVEO1WKDACK
CLIENT_SECRET:0UFI4D1UXNX50NIBJKGTLT4ML2JRA10ACFWGI4MP2JCCKQIV


### Explore the neighborhood


In [17]:
df_both.loc[0, 'Neighborhood']
neighborhood_latitude = df_both.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_both.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_both.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FP2YLWL5I4PV3ZST40Q2KFUSE5NP35FVVDP5APVEO1WKDACK&client_secret=0UFI4D1UXNX50NIBJKGTLT4ML2JRA10ACFWGI4MP2JCCKQIV&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eeb21dab4b684001b47f83c'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
Toronto_venues = getNearbyVenues(names=df_both['Neighborhood'],
                                   latitudes=df_both['Latitude'],
                                   longitudes=df_both['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [25]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2128, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [26]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
"Willowdale, Willowdale West",7,7,7,7,7,7
Woburn,3,3,3,3,3,3
Woodbine Heights,9,9,9,9,9,9


In [27]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


In [28]:
"Italian Restaurant" in Toronto_venues['Venue Category'].unique()

True

### Analyze Each Neighborhood

In [29]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
Toronto_onehot.shape

(2128, 275)

In [31]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Hei...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Man...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
Toronto_grouped.shape

(96, 275)

In [33]:
len(Toronto_grouped[Toronto_grouped["Italian Restaurant"] > 0])

24

In [61]:
# Dataframe for Italian Restaurant
Toronto_italian = Toronto_grouped[["Neighborhood","Italian Restaurant"]]
pd.set_option('display.max_rows', Toronto_italian.shape[0]+1)
print(Toronto_italian)


                     Neighborhood  Italian Restaurant
0                       Agincourt            0.000000
1          Alderwood, Long Branch            0.000000
2   Bathurst Manor, Wilson Hei...            0.000000
3                 Bayview Village            0.000000
4   Bedford Park, Lawrence Man...            0.086957
5                     Berczy Park            0.000000
6     Birch Cliff, Cliffside West            0.000000
7   Brockton, Parkdale Village...            0.041667
8   Business reply mail Proces...            0.000000
9   CN Tower, King and Spadina...            0.000000
10            Caledonia-Fairbanks            0.000000
11  Canada Post Gateway Proces...            0.000000
12                      Cedarbrae            0.000000
13             Central Bay Street            0.059701
14                       Christie            0.058824
15           Church and Wellesley            0.000000
16  Clarks Corners, Tam O'Shan...            0.076923
17  Cliffside, Cliffcrest, S

In [56]:
Toronto_italian.shape

(96, 2)

In [57]:
#Toronto_italian.to_excel("Toronto_italian.xlsx")
Toronto_italian.to_excel(r'D:\PC\CV\Learning\Modules\Toronto_italian.xlsx', index = False, header=True)

### Cluster Neighborhoods

In [36]:
# set number of clusters
kclusters = 3

Toronto_grouped_clustering = Toronto_italian.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 0, 1, 0, 0], dtype=int32)

In [37]:
#df_both.head()

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Toronto_merged = Toronto_italian.copy()

# add clustering labels
Toronto_merged["Cluster Labels"] = kmeans.labels_

In [38]:
Toronto_merged.rename(columns={"Neighborhood": "Neighborhood"}, inplace=True)
Toronto_merged.head()

#neighborhoods_venues_sorted.head()

,Neighborhood,Italian Restaurant,Cluster Labels
0,Agincourt,0.000000,0
1,"Alderwood, Long Branch",0.000000,0
2,"Bathurst Manor, Wilson Hei...",0.000000,0
3,Bayview Village,0.000000,0
4,"Bedford Park, Lawrence Man...",0.086957,2


In [39]:
Toronto_merged2 = Toronto_merged.join(Toronto_venues.set_index("Neighborhood"), on="Neighborhood")
print(Toronto_merged2.shape)
Toronto_merged2



(2128, 9)


,Neighborhood,Italian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,0.0,0,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
0,Agincourt,0.0,0,43.794200,-79.262029,Twilight,43.791999,-79.258584,Lounge
0,Agincourt,0.0,0,43.794200,-79.262029,El Pulgarcito,43.792648,-79.259208,Latin American Restaurant
0,Agincourt,0.0,0,43.794200,-79.262029,Mark's,43.791179,-79.259714,Clothing Store
0,Agincourt,0.0,0,43.794200,-79.262029,Commander Arena,43.794867,-79.267989,Skating Rink
...,...,...,...,...,...,...,...,...,...
94,York Mills West,0.0,0,43.752758,-79.400049,Tournament Park,43.751257,-79.399717,Park
94,York Mills West,0.0,0,43.752758,-79.400049,Yorkminster Park,43.755489,-79.401712,Park
94,York Mills West,0.0,0,43.752758,-79.400049,Jays Pub,43.752142,-79.405016,Bar
95,"York Mills, Silver Hills",0.0,0,43.757490,-79.374714,Windfields Cafeteria,43.755862,-79.370649,Cafeteria


In [40]:
# sort the results by Cluster Labels
print(Toronto_merged2.shape)
Toronto_merged2.sort_values(["Cluster Labels"], inplace=True)
Toronto_merged2



#Toronto_merged.dropna(subset = ["Cluster Labels"], inplace=True)

(2128, 9)


,Neighborhood,Italian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,0.000000,0,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
44,"Kingsview Village, St. Phi...",0.000000,0,43.688905,-79.554724,Subway,43.692936,-79.557351,Sandwich Place
43,"Kensington Market, Chinato...",0.000000,0,43.653206,-79.400049,The Supermarket,43.656680,-79.402954,Bar
43,"Kensington Market, Chinato...",0.000000,0,43.653206,-79.400049,Lettuce Knit,43.655014,-79.403342,Arts & Crafts Store
43,"Kensington Market, Chinato...",0.000000,0,43.653206,-79.400049,Pho Tien,43.652832,-79.406242,Vietnamese Restaurant
...,...,...,...,...,...,...,...,...,...
13,Central Bay Street,0.059701,2,43.657952,-79.387383,Jimmy's Coffee,43.658421,-79.385613,Coffee Shop
13,Central Bay Street,0.059701,2,43.657952,-79.387383,Tim Hortons,43.658570,-79.385123,Coffee Shop
13,Central Bay Street,0.059701,2,43.657952,-79.387383,Somethin' 2 Talk About,43.658395,-79.385338,Middle Eastern Restaurant
68,"Runnymede, Swansea",0.052632,2,43.651571,-79.484450,Tim Hortons,43.648526,-79.485066,Coffee Shop


In [41]:
Toronto_merged2['Cluster Labels'] =Toronto_merged['Cluster Labels'].astype(int)

In [42]:
Toronto_merged2.shape

(2128, 9)

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged2['Venue Latitude'], Toronto_merged2['Venue Longitude'], Toronto_merged2['Neighborhood'], Toronto_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow[cluster -1],
        fill=True,
        fill_color=rainbow[cluster -1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster1

In [48]:
Toronto_merged2.loc[Toronto_merged2['Cluster Labels'] == 0, Toronto_merged2.columns[[1] + list(range(5, Toronto_merged2.shape[1]))]]

,Italian Restaurant,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0.0,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
44,0.0,Subway,43.692936,-79.557351,Sandwich Place
43,0.0,The Supermarket,43.656680,-79.402954,Bar
43,0.0,Lettuce Knit,43.655014,-79.403342,Arts & Crafts Store
43,0.0,Pho Tien,43.652832,-79.406242,Vietnamese Restaurant
...,...,...,...,...,...
28,0.0,Aroma Espresso Bar,43.777700,-79.344652,Coffee Shop
28,0.0,Moxie's Classic Grill,43.777779,-79.343185,American Restaurant
28,0.0,Hudson's Bay,43.777830,-79.343043,Department Store
24,0.0,Price Chopper,43.739908,-79.512261,Grocery Store


### Cluster2


In [49]:
Toronto_merged2.loc[Toronto_merged2['Cluster Labels'] == 1, Toronto_merged2.columns[[1] + list(range(5, Toronto_merged2.shape[1]))]]

,Italian Restaurant,Venue,Venue Latitude,Venue Longitude,Venue Category
72,0.025641,Sukhothai,43.648487,-79.374547,Thai Restaurant
72,0.025641,D.W. Alexander,43.648333,-79.373826,Cocktail Bar
72,0.025641,Café Plenty,43.649118,-79.378313,Café
72,0.025641,Berczy Park,43.648048,-79.375172,Park
72,0.025641,The Reservoir Lounge,43.648517,-79.374556,Jazz Club
...,...,...,...,...,...
31,0.030000,Victoria's Secret PINK,43.653357,-79.380673,Lingerie Store
31,0.030000,Shisha&Co,43.656748,-79.374337,Smoke Shop
31,0.030000,GoodLife Fitness Toronto B...,43.653436,-79.382314,Gym
31,0.030000,Abercrombie & Fitch,43.652915,-79.380495,Clothing Store


### Cluster3


In [52]:
Toronto_merged2.loc[Toronto_merged2['Cluster Labels'] == 2, Toronto_merged2.columns[[1] + list(range(5, Toronto_merged2.shape[1]))]]

,Italian Restaurant,Venue,Venue Latitude,Venue Longitude,Venue Category
4,0.086957,Dino's No Frills,43.730699,-79.418535,Grocery Store
84,0.057143,Akai Sushi,43.662470,-79.404946,Sushi Restaurant
84,0.057143,Bakka Phoenix Books,43.662959,-79.402601,Bookstore
84,0.057143,Harbord House,43.662466,-79.405410,Bar
84,0.057143,Almond Butterfly,43.662836,-79.403365,Bakery
...,...,...,...,...,...
13,0.059701,Jimmy's Coffee,43.658421,-79.385613,Coffee Shop
13,0.059701,Tim Hortons,43.658570,-79.385123,Coffee Shop
13,0.059701,Somethin' 2 Talk About,43.658395,-79.385338,Middle Eastern Restaurant
68,0.052632,Tim Hortons,43.648526,-79.485066,Coffee Shop
